In [4]:
import time
import csv
import os
from pathlib import Path
import sys
sys.path.insert(0, '../')
import pycor 
from pycor import utils, korlang

stopwatch = utils.StopWatch()


model_size = 7000
outputpath = "../../output/" + str(model_size) + "/"
model_path = outputpath + "model/"

pycor.loadmodel(model_path)



Loading Model from ../../output/7000/model/
Load  ../../output/7000/model//heads.csv   소요시간: 1.135
Load  ../../output/7000/model//tails.csv   소요시간: 0.01


In [4]:
def listfiles(path):
    result_arr = []
    filenames = os.listdir(path)
    for filename in filenames:
        full_filename = os.path.join(path, filename)
        result_arr.append(full_filename)
    return result_arr
    
files = []
files.extend( listfiles('../../data/wiki') )
files.extend( listfiles('../samples/news') )
files.extend( listfiles('../../data/news') )
files.extend( listfiles('../../data/blotter') )
files.extend( listfiles('../../data/zdnet') )
files.extend( listfiles('../../data/ciobiz') )
files.extend( listfiles('../../data/NP') )

print (len(files), "files")

3552 files


In [9]:

yeTags = set(['EFN'])

def gettail(word):
    if word.bestpair:
#         return word.bestpair.tags
        return word.bestpair.tail.text
    return 'X'

def gettag(word):
    if word.bestpair:
        return word.bestpair.tags
    return 'X'

def gethead(word):
    if word.bestpair:
        return word.bestpair.head.text
    return 'X'


yheadsMap = {}
ytailsMap = {}
ytagsMap = {}

def handleHaed(index, word, words):
    yset = yheadsMap.get(word.bestpair.head)
    if yset is None:
        yset = []
        yheadsMap[word.bestpair.head] = yset

    if index > 1:
        yset.append( [gethead(words[index-1]),gethead(words[index-2])])
    else:
        yset.append( [gethead(words[index-1])] )
        
def handleTag(index, word, words):
    yset = ytagsMap.get(word.bestpair.head)
    if yset is None:
        yset = []
        ytagsMap[word.bestpair.head] = yset
    if index > 1:
        yset.append( [gettag(words[index-1]),gettag(words[index-2])])
    else:
        yset.append( [gettag(words[index-1])] )
                
def handleTail(index, word, words):
    yset = ytailsMap.get(word.bestpair.head)
    if yset is None:
        yset = []
        ytailsMap[word.bestpair.head] = yset
    if index > 1:
        yset.append( [gettail(words[index-1]),gettail(words[index-2])])
    else:
        yset.append( [gettail(words[index-1])] )
        
def analyzeYongEon(file):
    sentence_array, words_array = pycor.readfile(file)
    
    for words in words_array:
        for index, word in enumerate(words):
            if word.bestpair and len(yeTags & set(word.bestpair.tags))>0 and word.text.endswith('다'):
                handleHaed(index, word, words)
                handleTail(index, word, words)
                handleTag(index, word, words)


def writeFile(path, yMap):
    with open(path, 'w', encoding='utf-8') as file :
        writer = csv.writer(file)
        for head, tags in yMap.items():
            headText = head.text
            for tag in tags:
                row = [headText]
                row.extend(tag)
                writer.writerow(row)
        file.close()
                         
stopwatch.start()

docsize = 7000

for file in files[:docsize]:
    if file.endswith(".txt"):
        analyzeYongEon(file)

outputpath = "../../output/" + str(docsize) + "/"

writeFile(outputpath+'/yongeon-heads.csv',yheadsMap)
writeFile(outputpath+'/yongeon-tails.csv',ytailsMap)
writeFile(outputpath+'/yongeon-tags.csv',ytagsMap)


print("Loading ",docsize," Docs: " , stopwatch.secmilli() , "(", stopwatch.millisecstr(), "ms.)")

Loading  7000  Docs:  58s.770ms. ( 58,770 ms.)


In [24]:
# yheadsMap 
# ytailsMap  
# ytagsMap  

def fill(ylist, ymap):
    for head, tags in ymap.items():
        headText = head.text
        for tag in tags:
            row = [headText]
            row.extend(tag)
            ylist.append(row) 
        
tailsList = []
tagsList = []

fill(tailsList,ytailsMap)
fill(tagsList,ytagsMap)

matrix = []

for index in range(len(tailsList)):
    tail = tailsList[index]
    tag = tagsList[index]
    row = [tail[0]]
    row.extend(tail[1:])
    row.extend(tag[1:])
    matrix.append(row)

print(len(matrix))

181347


In [25]:
for row in matrix[0:100]:
    print(row)

['않', '지', '가', ['EFQ'], ['EFQ-ga']]
['않', '지', '여', ['EFQ'], ['JKV']]
['않', '지', '가', ['EFQ'], ['EFQ-ga']]
['않', '지', '이', ['EFQ'], ['JKS']]
['않', '지는', '가', ['JX-SO', 'EFQ'], ['EFQ-ga']]
['않', '지', '이라', ['EFQ'], ['JKP']]
['않', '지', '도', ['EFQ'], ['JX-SO']]
['않', '지', '가', ['EFQ'], ['EFQ-ga']]
['않', '지', '까지', ['EFQ'], ['JX']]
['않', '지', '', ['EFQ'], []]
['않', '지', '가', ['EFQ'], ['EFQ-ga']]
['않', '지', '에', ['EFQ'], ['JKB-TO']]
['않', '지', '인지는', ['EFQ'], ['ETM', 'JKP']]
['않', '지', '', ['EFQ'], []]
['않', '지', '에', ['EFQ'], ['JKB-TO']]
['않', '지', '가', ['EFQ'], ['EFQ-ga']]
['않', '지', '가', ['EFQ'], ['EFQ-ga']]
['않', '지', '을', ['EFQ'], ['JKO']]
['않', '지', '만은', ['EFQ'], ['JX-SO', 'JX']]
['않', '지', '을', ['EFQ'], ['ETM']]
['않', '지', '가', ['EFQ'], ['EFQ-ga']]
['않', '지', '도', ['EFQ'], ['JX-SO']]
['않', '지', '가', ['EFQ'], ['EFQ-ga']]
['않', '지', '을', ['EFQ'], ['JKO']]
['않', 'X', '이지만', 'X', ['EC-but', 'JKP']]
['않', '지', '이', ['EFQ'], ['JKS']]
['않', '지는', '에', ['JX-SO', 'EFQ'], ['JKB-TO']]
['않', '